<a href="https://colab.research.google.com/github/ssvadla/Bridge_Project/blob/main/Supervised_Light_XG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import time
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
train = pd.read_csv('/content/drive/My Drive/Bridge_Project/Dataset_1/ConterminousUSPre5StatesKnownTrain.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.columns

Index(['Unnamed: 0', 'Skew', 'Number_Spans', 'Length_Max_Span',
       'Main_Design_Type', 'Deck_Design_Type', 'Wearing_Surface',
       'Deck_Protection', 'ADT', 'ADTT', 'State', 'Structure_Number',
       'Latitude', 'Longitude', 'Deck_Condition', 'Superstructure_Condition',
       'Substructure_Condition', 'Days_MinLT_0F', 'Days_MaxGT_90F',
       'Avg_Wind_Speed', 'Peak_Wind_Speed', 'Precipitation', 'Reconstructed',
       'Age', 'Deck_Area'],
      dtype='object')

In [4]:
Excluded_columns = ['Unnamed: 0','State', 'Structure_Number', 'Latitude', 'Longitude', 'Superstructure_Condition', 'Substructure_Condition']

In [5]:
train = train.drop(columns = Excluded_columns)

In [6]:
train.shape

(41902, 18)

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train['Main_Design_Type_Encoded'] = le.fit_transform(train['Main_Design_Type'])
train['Deck_Design_Type_Encoded'] = le.fit_transform(train['Deck_Design_Type'])
train['Wearing_Surface_Encoded'] = le.fit_transform(train['Wearing_Surface'])
train['Deck_Protection_Encoded'] = le.fit_transform(train['Deck_Protection'])

In [8]:
import lightgbm as lgb
classifier_lgb = lgb.LGBMClassifier()

In [9]:
X = train.drop(columns=['Deck_Condition','Main_Design_Type','Deck_Design_Type','Wearing_Surface','Deck_Protection']) 


In [10]:
y = train['Deck_Condition']

In [11]:
X.shape

(41902, 17)

In [12]:
start = time.time()
classifier_lgb.fit(X, y)
end = time.time()
print(f"Runtime of the program is {end - start}")

Runtime of the program is 2.700289487838745


In [13]:
test = pd.read_csv('/content/drive/My Drive/Bridge_Project/Dataset_1/ConterminousUSPre5StatesKnownTest.csv')

In [14]:
test = test.drop(columns = Excluded_columns)

In [15]:
test['Main_Design_Type_Encoded'] = le.fit_transform(test['Main_Design_Type'])
test['Deck_Design_Type_Encoded'] = le.fit_transform(test['Deck_Design_Type'])
test['Wearing_Surface_Encoded'] = le.fit_transform(test['Wearing_Surface'])
test['Deck_Protection_Encoded'] = le.fit_transform(test['Deck_Protection'])

In [16]:
X_test = test.drop(columns=['Deck_Condition','Main_Design_Type','Deck_Design_Type','Wearing_Surface','Deck_Protection']) 


In [17]:
y_test = test['Deck_Condition']

In [18]:
X_test.shape

(13970, 17)

In [19]:
test_pred = classifier_lgb.predict(X_test)

In [20]:
from sklearn.metrics import classification_report
classification_report = classification_report(y_test,test_pred, digits=4)

In [21]:
print(classification_report)

              precision    recall  f1-score   support

           6     0.5006    0.5061    0.5033      3861
           7     0.5819    0.6014    0.5915      4453
          45     0.5878    0.5768    0.5822      2746
          89     0.7448    0.7089    0.7264      2910

    accuracy                         0.5926     13970
   macro avg     0.6038    0.5983    0.6009     13970
weighted avg     0.5945    0.5926    0.5934     13970



In [22]:
from xgboost import XGBClassifier

from sklearn.metrics import classification_report
classifier_xg = XGBClassifier()
start = time.time()
classifier_xg.fit(X, y)
end = time.time()
test_pred_xg = classifier_xg.predict(X_test)
classification_report_xg = classification_report(y_test,test_pred_xg, digits=4)
print(f"Runtime of the program is {end - start}")
print(classification_report_xg)

Runtime of the program is 10.457679033279419
              precision    recall  f1-score   support

           6     0.4741    0.5175    0.4949      3861
           7     0.5560    0.5706    0.5632      4453
          45     0.5740    0.5200    0.5457      2746
          89     0.7231    0.6704    0.6958      2910

    accuracy                         0.5668     13970
   macro avg     0.5818    0.5696    0.5749     13970
weighted avg     0.5717    0.5668    0.5685     13970



In [23]:
from sklearn.ensemble import RandomForestClassifier
classifier_rf = RandomForestClassifier(n_estimators = 350, criterion = 'gini', max_features = 'auto', random_state = 42)
start = time.time()
classifier_rf.fit(X, y)
end = time.time()
test_pred_rf = classifier_rf.predict(X_test)
classification_report_rf = classification_report(y_test,test_pred_rf, digits=4)
print(f"Runtime of the program is {end - start}")
print(classification_report_rf)

Runtime of the program is 33.764416456222534
              precision    recall  f1-score   support

           6     0.5511    0.5320    0.5414      3861
           7     0.6098    0.6512    0.6298      4453
          45     0.6255    0.6045    0.6148      2746
          89     0.7614    0.7412    0.7512      2910

    accuracy                         0.6278     13970
   macro avg     0.6369    0.6322    0.6343     13970
weighted avg     0.6282    0.6278    0.6277     13970



In [24]:
from sklearn import svm
classifier_svm = svm.SVC(decision_function_shape='ovo')
start = time.time()
classifier_svm.fit(X, y)
end = time.time()
test_pred_svm = classifier_svm.predict(X_test)
classification_report_svm = classification_report(y_test,test_pred_svm, digits=4)
print(f"Runtime of the program is {end - start}")
print(classification_report_svm)

Runtime of the program is 148.868008852005
              precision    recall  f1-score   support

           6     0.3724    0.0865    0.1404      3861
           7     0.3188    0.9349    0.4754      4453
          45     0.4615    0.0022    0.0043      2746
          89     0.0000    0.0000    0.0000      2910

    accuracy                         0.3223     13970
   macro avg     0.2882    0.2559    0.1550     13970
weighted avg     0.2952    0.3223    0.1912     13970



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
